## Проект бот-рифмоплет

### Инструменты:
- Python 3
- Jupyter Notebook
- HTML
- CSS
- Bootstrap
- JQuery
- Markdown
- Цепи Маркова
+ https://rustih.ru/dzhordzh-bajron/


### Начало работы: 

- поиск текстов для корпуса
- скачивание и обработка для дальнейшего обучения

кусочки кода:

In [ ]:
import bs4 as bs
import urllib.request

for page in self.soup.findAll("div", class_ = "nav-links"):
            page_links.append(str(page))
        
for page in page_links:
    start_point = page.find("Последняя", 0) - 20
            new_start_point = page.find("page", start_point) + len("page/")
            end_point = page.find("/", new_start_point)
            try:
                page_value = int(page[new_start_point:end_point])
                page_values.append(page_value)
            except:
                pass
min_page = 0
        for value in set(page_values):
            if value > min_page:
                min_page = value

        page_links = []
        for page_value in range(1, value + 1):
            page_links.append(self.original_link + "page/" + str(page_value) + "/")
            

In [ ]:
        import bs4 as bs
        import urllib.request
        import time
        import random
        import io
        
        self._getPoems()
        all_text = []
        
        for idx, link in enumerate(self.poem_links):
            rng = random.randint(5, 15)
            time.sleep(rng)
            
            page_source = urllib.request.urlopen(link).read()
            soup = bs.BeautifulSoup(page_source, "lxml")
            
            for text in soup.findAll("div", class_ = "entry-content poem-text"):
                all_text.append(text.text)
                
        self.all_texts = all_text
        
        
        file_name = "poems " + self.author + ".txt"
        with io.open(file_name, "w", encoding="utf-8") as output:
            for poem in self.all_texts:
                output.write(poem + "\n")
                
                
                
        import io
        import re

        # чистим текст
        regex = re.compile("[^абвгдеёжзийклмнопрстуфхцчшщъыьэюя]")
        collector = []
        poems_file = io.open(self.original_file, "r", encoding="utf-8")

        if poems_file.mode == "r":
            contents = poems_file.readlines()
            for content in contents:
                temp_collector = []
                for word in content.split():
                    new_word = regex.sub("", word.lower())
                    if new_word != "" and new_word != " ":
                        temp_collector.append(new_word)
                collector.append(temp_collector)

        self.new_file = self.original_file[:-4] + "_clean.txt"

        with io.open(self.new_file, "w", encoding="utf-8") as output:
            temp_text = " "
            for entry in collector:
                new_poem = temp_text.join(entry)
                output.write(new_poem + "\n")

## Цепи

метод "Цепей Маркова" является частично случайной и создание новых слова в последовательности зависит от того, какие слова использовались ранее

- первые два слова стиха выбираются абсолютно случайно из всех возможных комбинаций слов
- создаёт список всех комбинаций в которых использовались именно 2 предыдущих слова и оценивает какое слово может идти за ними
- процесс повторяется, только теперь алгоритм оценивает все комбинации слов
- файл MarkovChains.py

## Проблемы:

1. Поиск рифм.
Рассматривались различные варианты кодирования рифм, но я остановилась на форматах aabb, где a - предложения, формирующие первую рифму, b - предложения, формирующую вторую рифму и т.д. и свободный порядок, но гласное - наличие схожих букв в конце каких-то предложений.

2. Технические шоколадки.
    - Касательно формата aabb -
Был найден Гутенберг корпус с поэзией, были разбиты предложения по рифмам aabb, abab, abba. Но для их обработки и обучения модели использовался PyTorch и, следовательно, CUDA. CUDA - это боль. Очень долго выдавало ошибку отсутсвия поддержкиб потом, на этапе тренировки модели, выпала фатальная ошибка - несовместимость с моей видеокартой. Так, эта неплохая идея с рифмами не успела реализоваться.

    - Попытки обработки при помощи T5 -
Опять же, ограничение технических возможностей.

## Наработки (изначальный план):

- Рифма
Изначально была идея брать на вход текст (слово), используя сайт, определить ударение, например груша->грУша, затем при помощи RusVectores|Сайта синонимов искать близкие по смыслу слова и отбирать среди них слова с окончанием уша/юша.
Определение рифмы в моем случае будет таким: созвучие заударной части слов и частичное совпадение самой ударной гласной. Т.е. кр[Ышка] - м[Ышка], зап[Ястье] - нен[Астье], [Ёлка] - мет[Ёлка]. 
А так же в строке должно быть одинаковое количество гласных. Были бы добавлены исключения типа "нА реку", "пО лесу", т.е. одно ударение на слово и предлог (т.к. одно фонетическое слово)

In [4]:
from bs4 import BeautifulSoup as bsoup
from urllib import parse
import requests
import re

x = input()
URL = "http://где-ударение.рф/в-слове-{0}"


def find_accent(x):
    word = x.lower().strip()
    _url = URL.format(word)
    req = requests.get(_url)
    page = bsoup(req.text, "html.parser")
    if req.status_code == 200:
        acc_rule = page.findChild(attrs={"class": "rule"})
        result = re.search(r'[а-я]*<b>[А-ЯЁ]<\/b>[а-я]*', str(acc_rule))
        word_w_acc = result.group(0).replace("<b>", "").replace("</b>", "")
        return word_w_acc
    return "Nope"

word = x
if x == x.lower():
    word = find_accent(x)
    print(word)
else:
    pass

груша
грУша


In [5]:
v = ["а", "о", "э", "и", "ы", "у", "ю", "я", "е", "ё"]
v_count = 0
for letter in word:
    if letter.lower() in list(v):
        v_count += 1
print("количество слогов:",v_count)

количество слогов: 2


- Генерация предложений
Была идея создать несколько паттернов, описывающих структуру предложения, где V - verb, N - noun, PP - preposition, Conj - conjunctive, типа: 
    - N V PP N N
    - Conj N V N

Мама купила на рынке ложку, а папа разбил плошку

Для этого были использованы модули gensim и mystem.

In [9]:
#с сайта


def get_words(page):
    list_of_syn = []
    element = page.findChild(attrs={"class": "synonyms-table"})
    result = re.findall(r'[а-я]*<\/a>', str(element))
    for w in result:
        w = w.replace("</a>", "")
        list_of_syn.append(w)
    return list_of_syn

def get_synonyms(word):
    URL = "https://synonyms.su/"
    word = word.strip().lower()
    _url = URL + word[0].upper() + "/" + word
    req = requests.get(_url)
    page = bsoup(req.text, "html.parser")
    words = ", ".join(get_words(page))
    return words

get_synonyms(word)

'растение, имя, фрукт, лампочка, вантуз, аграфена, азароль, дюшес, амбрет, дуля, агриппина, глива, бера, плюкалка, розоцветные, розоцветный, русле, бергамот, бере, бессемянка, дюшеска, малгоржатка, берре, виргулез, грушина, дулина, тыркалка, грушица, грушка, деканка, тонковетка, крассана, кюре, дерево, лимонка, клаппа, пневмогруша, семечковый'

In [ ]:
# gensim & mystem

import gensim
import logging
import urllib.request

import warnings
warnings.filterwarnings('ignore')

m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)
    
words = ['яблоко_S']

for word in words:
    # есть ли слово в модели? 
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

In [ ]:
from pymystem3 import Mystem

"""
Он токенизируется, лемматизируется и размечается по частям речи с использованием Mystem.
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).
"""


def tag_mystem(
    text="Мама родная!", mapping=None, postags=True
):
 

    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(",")[0]
            pos = pos.split("=")[0].strip()
            tagged.append(lemma.lower() + "_" + pos)
        except KeyError:
            continue  #пропускаю знаки препинания
    if not postags:
        tagged = [t.split("_")[0] for t in tagged]
    return tagged

print("in process")
m = Mystem()

print("input")
res = input()
output = tag_mystem(text=res)
word = " ".join(output)
if word in model:
    for i in model.most_similar(positive=[word]):
        print(i[0], i[1])
    print('\n')
else:
        # Увы!
    print('Увы, слова "%s" нет в модели!' % word)

Еще был найдет модуль генерации текста с начала слова и на основе цепей Маркова:

In [ ]:
from MarkovTextGenerator.markov_text_generator import MarkovTextGenerator



my_generator = MarkovTextGenerator(2, None)
my_generator.update("Есенин Сергей. Стихотворения. Поэмы - royallib.ru.txt")
my_generator.create_dump()

print(my_generator.start_generation("Мы"))#слово - то, с чего должно начинаться

Лучшее (и самое жизненное...), что он выдал:

Мы не можем, что с вами случилось? Хочется вырвать окно и убежать в луг.

### Почему не продожила наработки?

Идея создания паттернов показалась странной, тем более в процессе вылезли проблемы с форматами генсима и майстема - они разные. Позже я нашла код для их объединения. Слова-синонимы с сайта выглядят сомнительно, а как осуществить поиск по выдаче генсима определенной части речи для заполнения паттернов я не поняла.

По итогу, если мне дадут возможность, я бы доработала эту версию, ну а пока есть только [aRahna.pythonanywhere.com](aRahna.pythonanywhere.com )

## Спасибо за внимание!